In [24]:
from sqlite_db import thesisDB
import re
from csv import DictWriter

dbAddress = 'D:/Thesis/data.db'
output = 'D:/Thesis/corpus_lemma.csv'

db = thesisDB(dbAddress)

def writeLine(doc):
    return str(doc['docid'])+','+noComma(doc['headline'])+','+str(doc['date'])+','+noComma(doc['medium'])+','+doc['text']+'\n'

def noComma(string):
    if re.search(',',string):
        return '"'+string+'"'
    return string

In [25]:
#lemmatised csv corpus
corpus = dict()
document = []

current_docid = 1
for lemma,pos,docid in db.getTokens(returnColumns = ['lemma','pos','docid']):
    if pos == 'LET':
        continue
            
    if docid != current_docid:
        document.append(lemma)
        corpus[current_docid] = {'docid':current_docid,'text':' '.join(document)}
        document = []
        current_docid = docid
    else:
        document.append(lemma)

for docid,headline,date,medium in db.getDocuments(returnColumns = ['id','headline','date','medium']):
    if docid in corpus:
        corpus[docid].update({'headline':headline,'date':date,'medium':medium})

with open(output,'w+',encoding='utf-8') as f:
    writer = DictWriter(f,fieldnames = ['docid','headline','date','medium','text'])
    
    writer.writeheader()
    for _,doc in corpus.items():
        writer.writerow(doc)

del corpus,document,current_docid   